## Description:
这里基于电影推荐的数据集实现一下SVD算法， 看看效果

In [1]:
import random
import numpy as np
import pandas as pd

import math

## 读入数据
读取文件得到"用户-电影"的评分数据， 并且分为训练集和测试集， 这里的思想是首先给出数据存在的路径， 然后通过pandas读取数据， 然后遍历该数据集， 把相应的数据存放到字典中， 这里之所以会用字典， 是因为用户对电影的评分会存在大量的稀疏。 所以我们依然需要建立一个"{用户：{电影: 评分}}"的这样一个字典

In [2]:
data = pd.read_csv('ratings.csv')
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
# 声明两个字典， 分别是训练集和测试集
trainSet, testSet = {}, {}
trainSet_len, testSet_len = 0, 0
pivot = 0.75    # 训练集的比例

# 遍历data的每一行， 把userId, movidId, rating按照{user: {movidId: rating}}的方式存储， 当然定义一个随机种子进行数据集划分
for ele in data.itertuples():   # 遍历行这里推荐用itertuples， 比iterrows会高效很多
    user, movie, rating = getattr(ele, 'userId'), getattr(ele, 'movieId'), getattr(ele, 'rating')
    if random.random() < pivot:
        trainSet.setdefault(user, {})
        trainSet[user][movie] = rating
        trainSet_len += 1
    else:
        testSet.setdefault(user, {})
        testSet[user][movie] = rating 
        testSet_len += 1

print('Split trainingSet and testSet success!')
print('TrainSet = %s' % trainSet_len)
print('TestSet = %s' % testSet_len)

Split trainingSet and testSet success!
TrainSet = 75628
TestSet = 25208


## 建立模型
这里用基础的SVD试试， 也就是那个带正则和偏置的那个

In [5]:
class BasicSVD():
    def __init__(self, rating_data, F=5, alpha=0.1, lmbda=0.1, max_iter=100):
        self.F = F           # 这个表示隐向量的维度
        self.P = dict()          #  用户矩阵P  大小是[users_num, F]
        self.Q = dict()     # 物品矩阵Q  大小是[item_nums, F]
        self.bu = dict()   # 用户偏差系数
        self.bi = dict()    # 物品偏差系数
        self.mu = 0.0        # 全局偏差系数
        self.alpha = alpha   # 学习率
        self.lmbda = lmbda    # 正则项系数
        self.max_iter = max_iter    # 最大迭代次数
        self.rating_data = rating_data # 评分矩阵
        
        # 初始化矩阵P和Q, 方法很多， 一般用随机数填充， 但随机数大小有讲究， 根据经验， 随机数需要和1/sqrt(F)成正比
        cnt = 0    # 统计总的打分数， 初始化mu用
        for user, items in self.rating_data.items():
            self.P[user] = [random.random() / math.sqrt(self.F)  for x in range(0, F)]
            self.bu[user] = 0
            cnt += len(items) 
            for item, rating in items.items():
                if item not in self.Q:
                    self.Q[item] = [random.random() / math.sqrt(self.F) for x in range(0, F)]
                    self.bi[item] = 0
        self.mu /= cnt
        
    # 有了矩阵之后， 就可以进行训练, 这里使用随机梯度下降的方式训练参数P和Q
    def train(self):
        for step in range(self.max_iter):
            for user, items in self.rating_data.items():
                for item, rui in items.items():
                    rhat_ui = self.predict(user, item)   # 得到预测评分
                    # 计算误差
                    e_ui = rui - rhat_ui
                    
                    self.bu[user] += self.alpha * (e_ui - self.lmbda * self.bu[user])
                    self.bi[item] += self.alpha * (e_ui - self.lmbda * self.bi[item])
                    # 随机梯度下降更新梯度
                    for k in range(0, self.F):
                        self.P[user][k] += self.alpha * (e_ui*self.Q[item][k] - self.lmbda * self.P[user][k])
                        self.Q[item][k] += self.alpha * (e_ui*self.P[user][k] - self.lmbda * self.Q[item][k])
                    
            self.alpha *= 0.1    # 每次迭代步长要逐步缩小
    
    # 预测user对item的评分， 这里没有使用向量的形式
    def predict(self, user, item):
        return sum(self.P[user][f] * self.Q[item][f] for f in range(0, self.F)) + self.bu[user] + self.bi[item] + self.mu   

In [7]:
basicsvd = BasicSVD(trainSet, F=128)
basicsvd.train()

## 产生推荐列表， 并评估结果

In [18]:
# 这里产生推荐列表， 遍历物品列表， 如果用户看了， 那么就跳过， 否则， 预测用户对该电影的打分， 然后记录， 最后排名
movie_list = []
for user, items in trainSet.items():
    for item in items.keys():
        if item not in movie_list:
            movie_list.append(item)


def recommend(aim_user, n=10):
    rank = {}
    watched_movies = trainSet[aim_user] # 目标用户看过的电影
    
    for movie in movie_list:
        if movie in watched_movies:
            continue
        
        # 如果当前用户没看过， 就预测打分， 并保存到rank
        rank[movie] = basicsvd.predict(aim_user, movie)
        
    
    return sorted(rank.items(), key=lambda x: x[1], reverse=True)[:n] 

In [19]:
# 产生推荐列表
recommend(2)

[(14, 6.912961297646142),
 (28, 6.896711491633388),
 (247, 6.627213206884909),
 (246, 6.443379973996613),
 (162, 6.370138897598569),
 (176, 6.359571028111279),
 (18, 6.307755110893343),
 (101, 6.306581248928255),
 (69, 6.19060919674639),
 (272, 6.180703380315263)]

In [21]:
# 准确率、召回率和覆盖率
n = 10     # 推荐用户评分高的前10部
hit = 0
rec_count = 0     # 统计推荐的影片数量， 计算查准率
test_count = 0    # 统计测试集的影片数量， 计算查全率
all_rec_movies = set()    # 统计被推荐出来的影片个数， 无重复了， 为了计算覆盖率
item_populatity = dict()   # 计算新颖度

# 先计算每部影片的流行程度
for user, items in trainSet.items():
    for item in items.keys():
        if item not in item_populatity:
            item_populatity[item] = 0
        item_populatity[item] += 1    # 这里统计训练集中每部影片用户观看的总次数， 代表每部影片的流行程度


# 计算评测指标
ret = 0
ret_cou = 0
for user, items in trainSet.items():    # 这里得保证测试集里面的用户在训练集里面才能推荐
    
    test_movies = testSet.get(user, {})
    rec_movies = recommend(user)
    for movie, w in rec_movies:
        if movie in test_movies:
            hit += 1
        all_rec_movies.add(movie)
        ret += math.log(1+item_populatity[movie])
        ret_cou += 1
    rec_count += n
    test_count += len(test_movies)
    
    
precision = hit / (1.0 * rec_count)
recall = hit / (1.0 * test_count)
coverage = len(all_rec_movies) / len(item_populatity)
ret /= ret_cou*1.0
    
print('precisioin = %.4f\nrecall = %.4f\ncoverage = %.4f\npopularity = %.4f' % (precision, recall, coverage, ret))

NameError: name 'movie_count' is not defined

precisioin = 0.0072
recall = 0.0017
coverage = 0.0022
popularity = 2.7131
